In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Tải các thư viện cần thiết cho việc cào dữ liệu

In [1]:
!pip install selenium

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.2/467.2 kB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.2 MB/s eta 0:00:00


In [2]:
!apt-get update
!apt-get install chromium chromium-driver

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:7 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [2,228 kB]
Get:8 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease [18.1 kB]
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [812 kB]
Get:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:11 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,077 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,739 

# Import thư viện cần thiết

In [3]:
import numpy as np
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
import requests
import csv
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Tối ưu chrome cho việc cào dữ liệu 

In [4]:
def web_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--verbose")
    options.add_argument('--no-sandbox')
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')
    options.add_argument("--window-size=1920, 1200")
    options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(options=options)
    return driver

# Lấy đường link của trang web và sản phẩm sau đó lưu lại 

In [ ]:
opt = webdriver.ChromeOptions()
driver = webdriver.Chrome(options=opt)
driver.get('https://bibomart.com.vn/?gad_source=1&gclid=CjwKCAjww_iwBhApEiwAuG6ccIihHcZYV4fa4NwLwKC5jvbiz525nxdtOCYmXEjv5ACjOspiGnX40BoC1U0QAvD_BwE')

In [ ]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [ ]:
menu_items = soup.find_all('li', class_='menu megamenu-item has-child has-content') #Lấy tag li mà tên vs tên class = ""
urls = [item.a['href'] for item in menu_items if item.a] #Lấy ra url của từng trang sản phẩm một cách chuyên biệt

In [ ]:
urls

['https://bibomart.com.vn/be-uong-sua.html',
 'https://bibomart.com.vn/be-an-dam.html',
 'https://bibomart.com.vn/be-tam-va-ve-sinh.html',
 'https://bibomart.com.vn/be-mac.html',
 'https://bibomart.com.vn/be-khoe-va-an-toan.html',
 'https://bibomart.com.vn/be-choi-va-hoc.html',
 'https://bibomart.com.vn/be-ngu.html',
 'https://bibomart.com.vn/be-di-choi-va-van-dong.html',
 'https://bibomart.com.vn/danh-cho-me.html',
 'https://bibomart.com.vn/do-dung-trong-nha.html']

In [ ]:
# Lưu tất cả url vào một file csv
csv_file_path = 'urls.csv'
with open(csv_file_path, 'w', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(['URL'])
    for url in urls:
        csv_writer.writerow([url])

In [ ]:
url_df = pd.read_csv('urls.csv')
url_df.iterrows()

<generator object DataFrame.iterrows at 0x000001BB9B69E350>

In [ ]:
for index, row in url_df.iterrows():
    url = row['URL']
    print(url)
    response = requests.get(url)

https://bibomart.com.vn/be-uong-sua.html
https://bibomart.com.vn/be-an-dam.html
https://bibomart.com.vn/be-tam-va-ve-sinh.html
https://bibomart.com.vn/be-mac.html
https://bibomart.com.vn/be-khoe-va-an-toan.html
https://bibomart.com.vn/be-choi-va-hoc.html
https://bibomart.com.vn/be-ngu.html
https://bibomart.com.vn/be-di-choi-va-van-dong.html
https://bibomart.com.vn/danh-cho-me.html
https://bibomart.com.vn/do-dung-trong-nha.html


In [ ]:
url_df = pd.read_csv('urls.csv')
all_product_links = []
for index, row in url_df.iterrows():
    url = row['URL']
    while True:
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        product_links = [link['href'] for link in soup.find_all('a', class_='product photo product-item-photo')] 
        all_product_links.extend(product_links)
        next_page_link = soup.find('li', class_='item pages-item-next')
        if next_page_link:
            url = next_page_link.a['href']
        else:
            break

In [ ]:
len(all_product_links)

5182

In [ ]:
csv_file_path = 'Link_sanPham.csv'
with open(csv_file_path, 'w', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(['URL'])
    for url in all_product_links:
        csv_writer.writerow([url])

In [1]:
all_product_links = pd.read_csv('/content/drive/MyDrive/TTAI/Link_sanPham.csv')
all_product_links = all_product_links['URL'].to_list()

# Code để lấy thông tin sản phẩm

- Do việc lấy dữ liệu có vấn đề nên việc cào dữ liệu sẽ chia làm 2 phần:
    + Phần đầu sẽ lấy tên và giá
    + Phần sau sẽ lấy thông tin sản phẩm sau đó ghép lại thành dataframe hoản chỉnh

In [3]:
def scrape_product(product_url):
    driver = web_driver()
    driver.get(product_url)
    try:
        try:
            continue_button = WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="yesbutton"]')))
            continue_button.click()
            WebDriverWait(driver, 2).until(EC.invisibility_of_element((By.XPATH, '//*[@id="bio_ep"]')))
        except:
            pass
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        product_name = soup.find('h1', class_='page-title')
        product_name = product_name.text.strip() if product_name else None

        product_price = soup.find('span', class_='price')
        product_price = product_price.text.strip() if product_price else None
        return product_name,product_price
    finally:
        driver.quit()

In [7]:
def scrape_product(product_url):
    driver = web_driver()
    driver.get(product_url)
    try:
        try:
            continue_button = WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="yesbutton"]')))
            continue_button.click()
            WebDriverWait(driver, 2).until(EC.invisibility_of_element((By.XPATH, '//*[@id="bio_ep"]')))
        except:
            pass

        soup = BeautifulSoup(driver.page_source, 'html.parser')

        try:
            continue_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="tab-label-additional"]')))
            continue_button.click()

            soup = BeautifulSoup(driver.page_source, 'html.parser')

            table = soup.find('table', class_='data table additional-attributes')
            if table:
                table_data = []
                for row in table.find_all('tr'):
                    row_data = [col.text.strip() for col in row.find_all('td')]
                    table_data.append(row_data)
                return table_data
            else:
                return None
        except:
            pass
    finally:
        driver.quit()

In [2]:
product_data = []
for url in urls:
    product_name, product_price = scrape_product(url)
    if product_name and product_price:
        product_data.append([product_name, product_price])

In [10]:
product_info = []
for url in all_product_links:
    table_data = scrape_product(url)
    table_data = table_data if table_data else []
    product_info.append([table_data])

In [11]:
filename = "/content/drive/MyDrive/TTAI/SanPhamBio.csv"
with open(filename, 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Product Name', 'Product Price','Table Info'])
    writer.writerows([product_data, product_info])